In [30]:
%load_ext autoreload
%autoreload 2

In [7]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, RegexpTokenizer
from collections import Counter

from gensim.models import Word2Vec
import gensim.downloader as api

import pandas as pd
import numpy as np

import json
import os
from os.path import join, exists, dirname

from LexAI.database import Database
from LexAI.analyse import Analyse

In [8]:
data = Database()
analyse = Analyse()

In [24]:
import inspect
inspect.getfullargspec(data.client.get_indexes)

FullArgSpec(args=['self'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [10]:
[idx.get('name', None) for idx in data.client.get_indexes()]

['twitter_query',
 'twitter_politicians',
 'eurlex',
 'synonyms',
 'twitter_press',
 'consultations']

In [166]:
words = analyse.get_words()

{'europa': ['portugal',
  'Bucarest',
  'seria',
  'bayern',
  'città',
  'pirlo',
  'ribery',
  'nico',
  'benzema',
  'giornata'],
 'author': ['coauthor',
  'Author',
  'authors',
  'bestselling_author',
  'article_Anatoly_Yablokov',
  'co_authored',
  'coauthored',
  'novelist',
  "Hazel_Lodevico_To'o_Email",
  'Giuseppe_Ungaro_Email'],
 'industrial': ['Industrial',
  'shredded_warehouses',
  'biothreat_markets',
  'Yokohama_Isogo_area',
  'manufacturing',
  'MagIndustries_resource',
  'manu_facturing',
  'Smoke_ascends',
  'agricultural',
  'Kamaishi_crumpled_cars'],
 'sky': ['skies',
  'Lightning_flickered',
  'heavens',
  'blue_sky',
  'Flares_lit',
  'clouds',
  'darkening_sky',
  'starry_sky',
  'clouds_scudding_across',
  'fireball_streaking'],
 'short': ['Short',
  'long',
  'shorter',
  'WEGENER_focuses',
  'shortest',
  'Pattern_Trapper_ADVANCED_Short',
  'shortterm',
  'Callidus_SPM_software',
  'term',
  'andlong'],
 'majority': ['overwhelming_majority',
  'vast_majority'

# Requests query

In [216]:
data.client.index('twitter_politicians').get_all_update_status()[-5:]

[{'status': 'processed',
  'updateId': 58,
  'type': {'name': 'Settings',
   'settings': {'ranking_rules': {'Update': ['Typo',
      'Words',
      'Proximity',
      'Attribute',
      'WordsPosition',
      'Exactness',
      {'Desc': 'timestamp'}]},
    'distinct_attribute': 'Nothing',
    'primary_key': 'Nothing',
    'searchable_attributes': 'Nothing',
    'displayed_attributes': 'Nothing',
    'stop_words': 'Nothing',
    'synonyms': 'Nothing',
    'attributes_for_faceting': 'Nothing'}},
  'duration': 16.208538198,
  'enqueuedAt': '2021-06-08T12:32:09.189464461Z',
  'processedAt': '2021-06-08T12:32:52.945106957Z'},
 {'status': 'processed',
  'updateId': 59,
  'type': {'name': 'DocumentsPartial', 'number': 18996},
  'duration': 32.469146208,
  'enqueuedAt': '2021-06-08T12:32:10.711907902Z',
  'processedAt': '2021-06-08T12:33:46.034015360Z'},
 {'status': 'processed',
  'updateId': 60,
  'type': {'name': 'Settings',
   'settings': {'ranking_rules': {'Update': ['Typo',
      'Words',

In [241]:
import requests

url = 'http://35.223.18.2/indexes/eurlex/search'
key = 'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'
params = {'q': 'agriculture', 'limit':999999999}
headers = {'X-Meili-API-Key': key}
r = requests.get(url, params=params, headers=headers).json()
r

5741

In [237]:
filepath = join('data.json', 'twitter_politicians.json')
with open(filepath, 'r') as file:
    documents = json.load(file)
type(documents[0]['text_en'])

NoneType